In [95]:
pip install requests

In [96]:
import pandas as pd
import requests

In [97]:
df = pd.read_csv("collegiate_athlete_injury_dataset.csv")

In [98]:
df

,Athlete_ID,Age,Gender,Height_cm,Weight_kg,Position,Training_Intensity,Training_Hours_Per_Week,Recovery_Days_Per_Week,Match_Count_Per_Week,Rest_Between_Events_Days,Fatigue_Score,Performance_Score,Team_Contribution_Score,Load_Balance_Score,ACL_Risk_Score,Injury_Indicator
0,A001,24,Female,195,99,Center,2,13,2,3,1,1,99,58,100,4,0
1,A002,21,Male,192,65,Forward,8,14,1,3,1,4,55,63,83,73,0
2,A003,22,Male,163,83,Guard,8,8,2,1,3,6,58,62,100,62,0
3,A004,24,Female,192,90,Guard,1,13,1,1,1,7,82,74,78,51,0
4,A005,20,Female,173,79,Center,3,9,1,2,1,2,90,51,83,49,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,A196,23,Female,169,88,Guard,3,11,3,4,1,3,77,94,100,39,0
196,A197,21,Male,185,95,Forward,8,5,3,4,2,5,56,61,100,56,0
197,A198,19,Female,193,89,Center,5,6,2,1,1,5,81,81,100,25,0
198,A199,19,Male,166,55,Center,9,7,2,3,1,9,60,81,97,61,0


In [99]:
columns_to_keep = [
    'Athlete_ID', 'Age', 'Gender', 'Training_Intensity', 'Training_Hours_Per_Week',
    'Recovery_Days_Per_Week', 'Match_Count_Per_Week', 'Rest_Between_Events_Days',
    'Fatigue_Score', 'Load_Balance_Score', 'Injury_Indicator'
]
df = df[columns_to_keep].copy()

In [100]:
df

,Athlete_ID,Age,Gender,Training_Intensity,Training_Hours_Per_Week,Recovery_Days_Per_Week,Match_Count_Per_Week,Rest_Between_Events_Days,Fatigue_Score,Load_Balance_Score,Injury_Indicator
0,A001,24,Female,2,13,2,3,1,1,100,0
1,A002,21,Male,8,14,1,3,1,4,83,0
2,A003,22,Male,8,8,2,1,3,6,100,0
3,A004,24,Female,1,13,1,1,1,7,78,0
4,A005,20,Female,3,9,1,2,1,2,83,0
...,...,...,...,...,...,...,...,...,...,...,...
195,A196,23,Female,3,11,3,4,1,3,100,0
196,A197,21,Male,8,5,3,4,2,5,100,0
197,A198,19,Female,5,6,2,1,1,5,100,0
198,A199,19,Male,9,7,2,3,1,9,97,0


In [101]:
def generate_recovery_prompt(row):
    return f"""
Recovery Profile:
- Age: {row['Age']}
- Gender: {row['Gender']}
- Training Intensity: {row['Training_Intensity']}
- Training Hours Per Week: {row['Training_Hours_Per_Week']}
- Recovery Days Per Week: {row['Recovery_Days_Per_Week']}
- Match Count Per Week: {row['Match_Count_Per_Week']}
- Rest Between Events (Days): {row['Rest_Between_Events_Days']}
- Fatigue Score: {row['Fatigue_Score']}
- Load Balance Score: {row['Load_Balance_Score']}
- Injury Indicator: {row['Injury_Indicator']}

Given this profile, please provide a personalized recommendation balancing training intensity and recovery.
"""
    
df['prompt'] = df.apply(generate_recovery_prompt, axis=1)


In [102]:
print(df[['Athlete_ID', 'prompt']].head())

  Athlete_ID                                             prompt
0       A001  \nRecovery Profile:\n- Age: 24\n- Gender: Fema...
1       A002  \nRecovery Profile:\n- Age: 21\n- Gender: Male...
2       A003  \nRecovery Profile:\n- Age: 22\n- Gender: Male...
3       A004  \nRecovery Profile:\n- Age: 24\n- Gender: Fema...
4       A005  \nRecovery Profile:\n- Age: 20\n- Gender: Fema...


In [103]:
OPENROUTER_API_KEY = "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"  

def query_deepseek(prompt):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://yourdomain.com",
        "X-Title": "MyAthleteRecoveryApp"
    }

    payload = {
        "model": "deepseek/deepseek-r1-0528:free",
        "messages": [{"role": "user", "content": prompt}],
        "temperature": 0.7,
        "max_tokens": 512
    }

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        return f"Error: {response.status_code} - {response.text}"


print(query_deepseek("What is a good recovery plan for an athlete with high fatigue and low recovery days?"))
print(query_deepseek(test_prompt))

Here’s a comprehensive, step-by-step recovery plan designed for an athlete experiencing **high fatigue and low recovery days**. This plan prioritizes **restoration, adaptation, and sustainable progress**, using evidence-based strategies:

### Phase 1: Immediate Intervention (1-3 Days)  
**Goal:** **Reduce acute fatigue, reset the nervous system.**  
1. **Urgent Load Reduction:**  
   - Cut training volume **by 50–70%** for 1–3 days.  
   - Lower intensity: Keep efforts *below* aerobic threshold (Zone 1-2).  
   - Example: Replace hard sessions with 30–45 min light walking, cycling, or swimming.  

2. **Prior


In [105]:
test_prompt = "Suggest a light recovery plan for a 22-year-old athlete with high training intensity and moderate fatigue."
print(query_deepseek(test_prompt))


Here’s a targeted **light recovery plan** designed for a 22-year-old athlete managing high training intensity and moderate fatigue. This plan prioritizes restoring energy, preventing burnout, and enhancing adaptation *without* disrupting training momentum. Implement for **1-3 weeks** or until fatigue subsides
